# Uplift Model Figures

In [1]:
# Imports
import os
import shutil

import cv2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyvista as pv

import cmcrameri.cm as cmc

pv.start_xvfb()

from tqdm import tqdm

from gdtchron import calculate,plot,aft

In [ ]:
# Model setup figure
fig,axs = plt.subplots(1,2,dpi=300,figsize=(7,2))
file = 'meshes_tchron_uplift/meshes_tchron_uplift_000.vtu'
clim = (273,873)
cmap = cmc.lipari

plot.plot2D(file,'T',bounds=[-2,102,-2,22],ax=axs[0],
          cmap=cmap,colorbar=False,clim=clim)

axs[0].set_title('Temperature Structure',fontsize=10)
axs[0].set_xlabel('X Position (km)',fontsize=8)
axs[0].set_ylabel('Y Position (km)',fontsize=8)

times = [0,10,10,15,15,20]
y_vel = [0,0,10,10,0,0]
axs[1].plot(times,y_vel,c='black')
axs[1].set_title('Boundary Velocity',fontsize=10)
axs[1].set_xlabel('Time (Myr)',fontsize=8)
axs[1].set_ylabel('Y Velocity (mm/yr)',fontsize=8)
axs[1].set_xticks([0,5,10,15,20])

for k,ax in enumerate(axs):
    ax.tick_params(labelsize=6)
    ax.annotate(chr(97+k) + ')',(0,1.1),xycoords='axes fraction',fontsize=10)

cax = plot.add_colorbar(fig,cmap=cmap,vmin=clim[0],vmax=clim[1],location=[0.15,0.05,0.2,0.05])
cax.tick_params(labelsize=6)
cax.set_title('Temperature (K)',fontsize=6,pad=2)

plt.tight_layout()
fig.savefig('uplift_setup.jpg',bbox_inches='tight')


In [2]:
# Set up parameters
bounds = [35,65,8,20] # km
times = [10,15,20] # Myr
labels = ['Before Uplift','After Uplift','After Quiescence']

ahe_cmap = cmc.lapaz_r
aft_cmap = cmc.lajolla_r
zhe_cmap = cmc.bamako_r
cat_cmap = cmc.batlowS

clim = [0,20]
bar=False

In [ ]:
# Results figure
fig,axs = plt.subplots(3,4,dpi=300,figsize=(7,5))

for k,time in enumerate(times):

    file_num = str(time*10).zfill(3)
    file = 'meshes_tchron_uplift/meshes_tchron_uplift_' + file_num + '.vtu'
    
    axs[k,0].set_title(str(round(time,0)) +' Myr - ' + labels[k],loc='left',fontsize=10)

    file_dir='meshes_tchron_uplift'
    file_num = str(time)
    
    plot.plot2D(file,'AHe',bounds=bounds,ax=axs[k,0],
              cmap=ahe_cmap,colorbar=bar,clim=clim)
    
    plot.plot2D(file,'AFT',bounds=bounds,ax=axs[k,1],
            cmap=aft_cmap,colorbar=bar,clim=clim)
    
    plot.plot2D(file,'ZHe',bounds=bounds,ax=axs[k,2],
        cmap=zhe_cmap,colorbar=bar,clim=clim)
    
    mesh = pv.read(file)

    y = np.round(mesh.points[:,1]/1000,0)
    AHe = mesh['AHe']
    AFT = mesh['AFT']
    ZHe = mesh['ZHe']

    df = pd.DataFrame({'y':y,'AHe':AHe,'AFT':AFT,'ZHe':ZHe})
    df_max = df.groupby('y').agg({'y':'first','AHe':'max','AFT':'max','ZHe':'max'})

    axs[k,3].plot(df_max['AHe'],df_max['y'],c=cat_cmap.colors[6],label='AHe')
    axs[k,3].plot(df_max['AFT'],df_max['y'],c=cat_cmap.colors[4],label='AFT')
    axs[k,3].plot(df_max['ZHe'],df_max['y'],c=cat_cmap.colors[2],label='ZHe')

    axs[k,3].set_xlim(-0.25,20)
    axs[k,3].set_ylim(bounds[2],bounds[3])
    axs[k,3].text(0.3,-.03,
                'Surface Ages\n'
                  + 'AHe: ' + str(round(df_max['AHe'][20],1)) + ' Ma\n'
                  + 'AFT: ' + str(round(df_max['AFT'][20],1)) + ' Ma\n'
                  + 'ZHe: '+ str(round(df_max['ZHe'][20],1)) + ' Ma\n',
                transform=axs[k,3].transAxes,fontsize=7)

cax_ahe = plot.add_colorbar(fig,cmap=ahe_cmap,vmin=clim[0],vmax=clim[1],location=[0.08,0.68,0.15,0.02])
cax_aft = plot.add_colorbar(fig,cmap=aft_cmap,vmin=clim[0],vmax=clim[1],location=[0.32,0.68,0.15,0.02])
cax_zhe = plot.add_colorbar(fig,cmap=zhe_cmap,vmin=clim[0],vmax=clim[1],location=[0.56,0.68,0.15,0.02])

for cax in [cax_ahe,cax_aft,cax_zhe]:
    cax.tick_params(labelsize=6)
cax_ahe.set_title('AHe Age (Ma)',fontsize=6)
cax_aft.set_title('AFT Age (Ma)',fontsize=6)
cax_zhe.set_title('ZHe Age (Ma)',fontsize=6)

axs[0,0].set_ylabel('Y Position (km)',fontsize=6)
axs[1,0].set_ylabel('Y Position (km)',fontsize=6)
axs[2,0].set_ylabel('Y Position (km)',fontsize=6)

axs[2,0].set_xlabel('X Position (km)',fontsize=6)
axs[2,1].set_xlabel('X Position (km)',fontsize=6)
axs[2,2].set_xlabel('X Position (km)',fontsize=6)
axs[2,3].set_xlabel('Maximum Age (Ma)',fontsize=6)

axs[0,3].set_ylabel('Y Position (km)',fontsize=6)
axs[1,3].set_ylabel('Y Position (km)',fontsize=6)
axs[2,3].set_ylabel('Y Position (km)',fontsize=6)

axs[0,3].legend(fontsize=6)

for ax in axs.flat:
    ax.tick_params(labelsize=6)

plt.tight_layout()
fig.savefig('uplift_results.jpg')